### Import Required Libraries and Set Up Environment Variables

In [53]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [54]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [55]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
#query_url = f"{url}api-key={nyt_api_key}&q={filter_query}&fq={filter_query}&sort={sort}&begin_date={begin_date}&end_date={end_date}&fl={field_list}"
# Reconstruct the query URL with URL encoding and appropriate parameters
query_params = {
    'api-key': nyt_api_key,
    'q': filter_query,
    'sort': sort,
    'begin_date': begin_date,
    'end_date': end_date,
    'fl': field_list
}
# Parameters for the API request
params = {
    'api-key': nyt_api_key,
    'q': '"love"',
    'fq': 'section_name:("Movies") AND type_of_material:("Review")',
    'sort': 'newest',
    'fl': 'headline,web_url,snippet,source,keywords,pub_date,byline,word_count'
}

#encoded_query_params = "&".join([f"{key}={value}" for key, value in query_params.items()])
#$query_url = f"{url,params}"
# [Your existing code for setting up the query URL]

# Make the API request
response = requests.get(url, params=params)
data = response.json()

# Check if there is valid data and print the first result
if data["response"]["docs"]:
    print("Valid data received from the API.")
    first_result = data["response"]["docs"][0]  # Retrieve the first document
    print("First result:")
    print(json.dumps(first_result, indent=4))   # Print the first result in JSON format
else:
    print("No valid data received. Check the query or API key.")



Valid data received from the API.
First result:
{
    "web_url": "https://www.nytimes.com/2023/12/07/movies/our-son-review-the-right-to-break-up.html",
    "snippet": "A simple yet engaging melodrama, starring Billy Porter and Luke Evans, explores what it means for two fathers to divorce.",
    "source": "The New York Times",
    "headline": {
        "main": "\u2018Our Son\u2019 Review: The Right to Break Up",
        "kicker": null,
        "content_kicker": null,
        "print_headline": "Our Son",
        "name": null,
        "seo": null,
        "sub": null
    },
    "keywords": [
        {
            "name": "subject",
            "value": "Movies",
            "rank": 1,
            "major": "N"
        },
        {
            "name": "persons",
            "value": "Evans, Luke (1979- )",
            "rank": 2,
            "major": "N"
        },
        {
            "name": "persons",
            "value": "Oliver, Bill (1968- )",
            "rank": 3,
            "major

In [56]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(3):
    # create query with a page number
    # API results show 10 articles at a time
    #query_url = f"{url}api-key={nyt_api_key}&q={filter_query}&fq={filter_query}&sort={sort}&begin_date={begin_date}&end_date={end_date}&fl={field_list}&page={page}"

    # Make a "GET" request and retrieve the JSON
    response = requests.get(url, params=params).json()

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    # Check if there are results
    print(f"Processing page {page} of results")

    # If no results are found, print the page number and break from the loop
    if not response["response"]["docs"]:
        print(f"No results found for page {page}")
        break

    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in response["response"]["docs"]:
            reviews_list.append(review)
    except KeyError:
        # Print the page number that had no results then break from the loop
        print(f"No results found for page {page}")
        break





Processing page 0 of results
Processing page 1 of results
Processing page 2 of results


In [57]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
from pandas import json_normalize

preview = json.dumps(reviews_list[:5], indent=4)
print(preview)


[
    {
        "web_url": "https://www.nytimes.com/2023/12/07/movies/our-son-review-the-right-to-break-up.html",
        "snippet": "A simple yet engaging melodrama, starring Billy Porter and Luke Evans, explores what it means for two fathers to divorce.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018Our Son\u2019 Review: The Right to Break Up",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "Our Son",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "persons",
                "value": "Evans, Luke (1979- )",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
                "valu

In [59]:
from pandas import json_normalize

# Convert reviews_list to a Pandas DataFrame using json_normalize()
df = json_normalize(reviews_list)

# Verify DataFrame creation
if df is not None:
    print("DataFrame created successfully.")

    # Check for a non-empty DataFrame
    if not df.empty:
        print(f"DataFrame contains {df.shape[0]} rows and {df.shape[1]} columns.")

        # Display basic information and the first few rows of the DataFrame
        print("Basic information about the DataFrame:")
        df.info()
        print("\nPreview of the first few rows:")
        print(df.head())
    else:
        print("DataFrame is empty.")
else:
    print("Failed to create DataFrame.")


DataFrame created successfully.
DataFrame contains 30 rows and 16 columns.
Basic information about the DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   web_url                  30 non-null     object
 1   snippet                  30 non-null     object
 2   source                   30 non-null     object
 3   keywords                 30 non-null     object
 4   pub_date                 30 non-null     object
 5   word_count               30 non-null     int64 
 6   headline.main            30 non-null     object
 7   headline.kicker          21 non-null     object
 8   headline.content_kicker  0 non-null      object
 9   headline.print_headline  30 non-null     object
 10  headline.name            0 non-null      object
 11  headline.seo             0 non-null      object
 12  headline.sub             0 non-null 

In [61]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
# Check if 'headline.main' column exists
if 'headline.main' in df.columns:
    # Extract the title using a lambda function
    df['title'] = df['headline.main'].apply(
        lambda x: x[x.find("\u2018")+1:x.find("\u2019 Review")] if "\u2018" in x and "\u2019 Review" in x else None
    )
else:
    print("Column 'headline.main' does not exist in the DataFrame.")

# First, print the DataFrame columns to check if 'headline.main' exists
print(df.columns)
#df['title'] = df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])



Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.person',
       'byline.organization', 'title'],
      dtype='object')


In [62]:
# Extract 'name' and 'value' from items in "keywords" column

        # Extract 'name' and 'value'
def extract_keywords(keywords_list):
    extracted_keywords = []
    for item in keywords_list:
        name = item.get('name', '')
        value = item.get('value', '')
        keyword_item = f"{name}: {value}"
        extracted_keywords.append(keyword_item)
    return extracted_keywords   
        # Append the keyword item to the extracted_keywords list
df['extracted_keywords'] = df['keywords'].apply(extract_keywords)



# Fix the "keywords" column by converting cells from a list to a string
df['keywords'] = df['extracted_keywords'].apply(', '.join)
df.drop(columns=['extracted_keywords'], inplace=True)

In [63]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
# Check if 'title' column exists in the DataFrame
if 'title' in df.columns:
    # Create a list from the "title" column
    titles_list = df['title'].to_list()
    print("Titles list created successfully.")
    print("First few titles:", titles_list[:5])  # Print the first few titles for verification
else:
    print("Column 'title' does not exist in the DataFrame.")


Titles list created successfully.
First few titles: ['Our Son', 'Concrete Utopia', 'Renaissance: A Film by Beyoncé', 'May December', 'American Symphony']


### Access The Movie Database API

In [64]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [69]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Define the "titles" variable
titles = []

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        print("Application is sleeping...")
        time.sleep(1)
    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    query_url = f"https://api.themoviedb.org/3/search/movie?query={title}&api_key={tmdb_api_key}"
    response = requests.get(query_url)
    movie_data = response.json()
    
   

    # Include a try clause to search for the full movie details.

    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
   
        # Get movie id
        movie_id = movie_data["results"][0]["id"]

        # Make a request for a the full movie details
        movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        response_details = requests.get(movie_details_url)
        movie_details = response_details.json()

        # Execute "GET" request with url
        title = movie_details['title']
        
        # Extract the genre names into a list
        genres = [genre['name'] for genre in movie_details['genres']]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang['english_name'] for lang in movie_details['spoken_languages']]
    

        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in movie_details['production_countries']]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_info = {
            'title': title,
            'genres': genres,
            'spoken_languages': spoken_languages,
            'production_countries': production_countries
        }
        
        tmdb_movies_list.append(movie_info)
        
        # Print out the title that was found
        print(f"Title '{title}' found.")
    except (IndexError, KeyError):
        print(f"Title '{title}' not found.")

In [70]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
from pandas import json_normalize

preview = json.dumps(reviews_list[:5], indent=4)
print(preview)

[
    {
        "web_url": "https://www.nytimes.com/2023/12/07/movies/our-son-review-the-right-to-break-up.html",
        "snippet": "A simple yet engaging melodrama, starring Billy Porter and Luke Evans, explores what it means for two fathers to divorce.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018Our Son\u2019 Review: The Right to Break Up",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "Our Son",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "persons",
                "value": "Evans, Luke (1979- )",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
                "valu

In [71]:
# Convert the results to a DataFrame
df = json_normalize(reviews_list)

### Merge and Clean the Data for Export

In [77]:
# Rename 'headline.main' column to 'title' in the reviews_list DataFrame
reviews_list.rename(columns={'headline.main': 'title'}, inplace=True)

# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_list, tmdb_movies_list, on='title')




AttributeError: 'list' object has no attribute 'rename'

In [78]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'production_countries', 'spoken_languages']

# Create a list of characters to remove
chars_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Loop through the list of characters to remove
    for char in chars_to_remove:
        # Remove the character
        df[column] = df[column].str.replace(char, '')
        
    # Convert the column to type 'str'
    df[column] = df[column].astype(str)
    # Loop through characters to remove
    for char in chars_to_remove:
        # Remove the character
        df[column] = df[column].str.replace(char, '')
for column in columns_to_fix:
    # Loop through the list of characters to remove
    for char in chars_to_remove:
        # Remove the character
        merged_df[column] = merged_df[column].str.replace(char, '')
        
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)

    # Loop through characters to remove
    for char in chars_to_remove:
    # Remove the character
        merged_df[column] = merged_df[column].str.replace(char, '')


# Display the fixed DataFrame
merged_df.head()



KeyError: 'genres'

In [17]:
# Drop "byline.person" column
merged_df.drop(columns=['byline.person'], inplace=True)


In [18]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates(inplace=True)



In [19]:
# Export data to CSV without the index
merged_df.to_csv('output/nyt_movie_reviews.csv', index=False)

